<a href="https://colab.research.google.com/github/Gau-6426/PCAP/blob/main/Code/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [3]:
df_train = pd.read_csv("https://raw.githubusercontent.com/Gau-6426/PCAP/main/Data/train.csv")
df_train.head(20)

,_id,segment_id,date,weekday,period,LOS,s_node_id,e_node_id,length,street_id,max_velocity,street_level,street_name,street_type,long_snode,lat_snode,long_enode,lat_enode
0,0,26,2021-04-16,4,period_0_30,A,366428456,366416066,116,32575820,NaN,4,Nguyễn Văn Bá,tertiary,106.768732,10.841506,106.769254,10.842422
1,1,33,2020-08-02,6,period_23_30,C,366469460,3792257828,26,32575862,NaN,3,Đường số 5,secondary,106.761957,10.878650,106.762143,10.878808
2,2,33,2020-08-03,0,period_0_00,D,366469460,3792257828,26,32575862,NaN,3,Đường số 5,secondary,106.761957,10.878650,106.762143,10.878808
3,3,67,2021-03-09,1,period_9_30,B,366403668,5755066033,7,32575862,NaN,3,Đường số 5,secondary,106.768412,10.880817,106.768461,10.880771
4,4,67,2021-03-23,1,period_9_30,B,366403668,5755066033,7,32575862,NaN,3,Đường số 5,secondary,106.768412,10.880817,106.768461,10.880771
5,5,70,2020-08-02,6,period_23_30,D,5816921541,4614188871,8,32575864,40.0,3,Châu Văn Liêm,secondary,106.659032,10.751940,106.659019,10.752017
6,6,70,2020-08-03,0,period_0_00,E,5816921541,4614188871,8,32575864,40.0,3,Châu Văn Liêm,secondary,106.659032,10.751940,106.659019,10.752017
7,7,71,2020-08-02,6,period_23_30,E,366441747,5738173336,43,32575869,NaN,4,Lê Văn Thịnh,unclassified,106.765737,10.776592,106.765964,10.776912
8,8,71,2020-08-03,0,period_0_00,C,366441747,5738173336,43,32575869,NaN,4,Lê Văn Thịnh,unclassified,106.765737,10.776592,106.765964,10.776912
9,9,161,2020-09-11,4,period_0_30,A,3202462316,3202462317,21,32575935,NaN,3,Tân Phú,primary_link,106.723620,10.733619,106.723813,10.733604
